In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



def print_res(path):
    data = pd.read_csv(path)

    fig, ax = plt.subplots()
    x = y = np.linspace(-5, 5, 100)
    X,Y = np.meshgrid(x,y)
    Z = (X ** 2 + Y - 11) ** 2 + (X + Y ** 2 - 7) ** 2
    # Z = -20*np.exp(-0.2*np.sqrt(0.5*(X**2+Y**2)))-np.exp(0.5*(np.cos(2*np.pi*X)+np.cos(2*np.pi*Y)))+20+np.exp(1)

    map = ax.contourf(X,Y,Z,cmap="plasma", levels=100)
    fig.colorbar(map)

    plt.scatter(data["a"],data["b"],c="cyan",s=0.1)
    plt.plot()

In [ ]:
from zellij.core import Loss, Threshold, Experiment, Minimizer
from zellij.core import ContinuousSearchspace, FloatVar, ArrayVar
from zellij.utils.neighborhoods import FloatInterval, ArrayInterval, Intervals
from zellij.strategies.mixed.genetic_algorithm import Steady_State_GA, Genetic_algorithm
from zellij.utils.operators import NeighborMutation, DeapTournament, DeapOnePoint
import numpy as np
from zellij.utils.benchmarks import himmelblau

lf = Loss(save=True)(himmelblau)

sp = ContinuousSearchspace(ArrayVar(
                          FloatVar("a",-5,5, neighbor=FloatInterval(0.5)),
                          FloatVar("b",-5,5,neighbor=FloatInterval(0.5)),
                          neighbor=ArrayInterval())
                        ,lf, neighbor=Intervals(),
                        mutation = NeighborMutation(1),
                        selection = DeapTournament(5),
                        crossover = DeapOnePoint())

stop = Threshold(lf, 'calls', 1000)
ga = Genetic_algorithm(sp, pop_size=30)
exp = Experiment(ga, stop, save="exp_ga", backup_interval=5)
exp.run()

print_res("./exp_ga/outputs/all_evaluations.csv")

In [ ]:
from zellij.core import Loss, Threshold, Experiment, Minimizer
from zellij.core import ContinuousSearchspace, FloatVar, ArrayVar
from zellij.utils.neighborhoods import FloatInterval, ArrayInterval, Intervals
from zellij.strategies.mixed.genetic_algorithm import Steady_State_GA, Genetic_algorithm
from zellij.utils.operators import NeighborMutation, DeapTournament, DeapOnePoint
import numpy as np
from zellij.utils.benchmarks import himmelblau

lf = Loss(save=True)(himmelblau)

sp = ContinuousSearchspace(ArrayVar(
                          FloatVar("a",-5,5, neighbor=FloatInterval(0.5)),
                          FloatVar("b",-5,5,neighbor=FloatInterval(0.5)),
                          neighbor=ArrayInterval())
                        ,lf, neighbor=Intervals(),
                        mutation = NeighborMutation(1),
                        selection = DeapTournament(5),
                        crossover = DeapOnePoint())

stop = Threshold(lf, 'calls', 1000)
ga = Steady_State_GA(sp, pop_size=30)
exp = Experiment(ga, stop, save="exp_sga", backup_interval=5)
exp.run()
print_res("./exp_sga/outputs/all_evaluations.csv")

In [ ]:
from zellij.core import ContinuousSearchspace, FloatVar, ArrayVar
from zellij.core import Loss, Threshold, Experiment
from zellij.utils.neighborhoods import FloatInterval, ArrayInterval, Intervals
from zellij.strategies import Simulated_annealing
from zellij.strategies.tools import MulExponential
from zellij.utils.benchmarks import himmelblau

lf = Loss(save=True)(himmelblau)
sp = ContinuousSearchspace(ArrayVar(
                          FloatVar("a",-5,5, neighbor=FloatInterval(0.5)),
                          FloatVar("b",-5,5,neighbor=FloatInterval(0.5)),
                          neighbor=ArrayInterval())
                        ,lf, neighbor=Intervals())

lf.save = False
x_start = [sp.random_point(1)]
x_start, y_start, _ = lf(x_start)
lf.save=True

cooling = MulExponential(0.85,100,2,3)
sa = Simulated_annealing(sp, cooling, 1, 5)


stop = Threshold(lf, 'calls', 1000)
exp = Experiment(sa, stop, save="exp_sa", backup_interval=5)

exp.run(x_start, y_start)
print_res("./exp_sa/outputs/all_evaluations.csv")

In [3]:
from zellij.core import ContinuousSearchspace, FloatVar, IntVar, CatVar, ArrayVar
from zellij.core import Loss, Threshold, Experiment, MockModel
from zellij.utils.converters import FloatMinmax, IntMinmax, CatMinmax, ArrayConverter, Basic
from zellij.strategies import Bayesian_optimization

from gpytorch.constraints import Interval
from gpytorch.kernels import MaternKernel
from zellij.utils.benchmarks import himmelblau

lf = Loss(save=True)(himmelblau)
sp = ContinuousSearchspace(ArrayVar(FloatVar("a",-5,5),FloatVar("b",-5,5)),lf)

covar_module = MaternKernel(nu=2.5, ard_num_dims=sp.size)
noise_constraint = Interval(1e-8, 1e-3)

bo = Bayesian_optimization(sp, covar_module=covar_module, noise_constraint=noise_constraint)

stop = Threshold(lf, 'calls', 1000)
exp = Experiment(bo, stop, save="exp_bo", backup_interval=5)

exp.run()
print_res("./exp_bo/outputs/all_evaluations.csv")

Saving folder already exists, exp_bo. Results will still be saved in it.


{'covar_module': MaternKernel(
  (raw_lengthscale_constraint): Positive()
)} {'noise_constraint': Interval(1.000E-08, 1.000E-03)} {} {}
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[0.9032, 3.9331]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[1.7867, 0.7706]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[ 4.6181, -1.7769]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[2.3162, 3.7964]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-2.3392, -3.3760]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-1.2251,  0.2642]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-0.1510,  0.1438]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-0.6967, -3.132

/home/tfirmin/venv/zellij/lib/python3.10/site-packages/botorch/fit.py:163: RuntimeWarning: All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
  warn(str(err), RuntimeWarning)


tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[1.4823, 1.5414]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-1.2778, -3.3014]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-3.5788,  2.9138]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[ 2.0942, -0.6536]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-2.0256,  2.5573]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-4.2401, -4.0884]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-3.1618, -0.6589]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[4.1499, 3.9093]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-4.1609, -2.1563]], dtype=torch.float

/home/tfirmin/venv/zellij/lib/python3.10/site-packages/botorch/fit.py:163: RuntimeWarning: All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
  warn(str(err), RuntimeWarning)


...

tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-1.8714,  1.7977]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-1.8782,  4.2483]], dtype=torch.float64)


/home/tfirmin/venv/zellij/lib/python3.10/site-packages/botorch/fit.py:163: RuntimeWarning: All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
  warn(str(err), RuntimeWarning)


tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-2.1301,  3.0281]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[ 1.4200, -4.2038]], dtype=torch.float64)


/home/tfirmin/venv/zellij/lib/python3.10/site-packages/botorch/fit.py:163: RuntimeWarning: All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
  warn(str(err), RuntimeWarning)


tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-2.0068, -0.0206]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-3.3220,  1.9244]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-2.2720,  3.3439]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[4.0813, 3.6381]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[ 2.0924, -1.7082]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[ 4.6748, -1.4768]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[-3.7435, -3.4633]], dtype=torch.float64)
tensor([[-5., -5.],
        [ 5.,  5.]], dtype=torch.float64)
tensor([[1.5162, 2.5827]], dtype=torch.float64)


In [ ]:
from zellij.core import ContinuousSearchspace, FloatVar, IntVar, CatVar, ArrayVar
from zellij.core import Loss, Threshold, Experiment, MockModel
from zellij.utils.converters import FloatMinmax, IntMinmax, CatMinmax, ArrayConverter, Basic
from zellij.strategies import TuRBO
from zellij.utils.benchmarks import himmelblau
from botorch.test_functions import Ackley

from gpytorch.constraints import Interval
from gpytorch.kernels import MaternKernel, ScaleKernel
import torch

fun = Ackley(dim=2, negate=True)
fun.bounds[0, :].fill_(-5)
fun.bounds[1, :].fill_(10)

def nfunc(x):
    x = torch.tensor(x)
    res = float(fun(x).item())
    return res

lf = Loss(save=True)(nfunc)

sp = ContinuousSearchspace(ArrayVar(
                          FloatVar("a",-5,10, converter=FloatMinmax()),
                          FloatVar("b",-5,10, converter=FloatMinmax()),
                          converter=ArrayConverter())
                        ,lf, converter=Basic())

covar_module = ScaleKernel(MaternKernel(nu=2.5, ard_num_dims=sp.size, lengthscale_constraint=Interval(0.005, 4.0)))
noise_constraint = Interval(1e-8, 1e-3)

bo = TuRBO(sp, batch_size=4, initial_size=40,covar_module=covar_module, noise_constraint=noise_constraint)

stop = Threshold(lf, 'calls', 1000)
exp = Experiment(bo, stop, save="exp_turbo", backup_interval=5)

exp.run()
print(f"Best solution:f({lf.best_point})={lf.best_score}")

In [ ]:
from zellij.strategies import ILS
from zellij.strategies.tools.geometry_new import Hypersphere
from zellij.utils.converters import FloatMinmax, FloatBinning, ArrayConverter, FloatBinary, FloatBinaryStochastic, ArrayBinaryBest, Basic

lf = Loss(save=True)(himmelblau)
values = ArrayVar(
    FloatVar("float_1", -5 , 5, converter=FloatMinmax()),
    FloatVar("float_2", -5, 5, converter=FloatMinmax()),
    converter=ArrayConverter(),
)
sp = Hypersphere(values, lf, converter=Basic())
phs = ILS(sp)